In [1]:
import time
from ore_algebra import *
DOP, t, D = DifferentialOperators()

# These may be overridden in input file, but need not be specified there
periodsOnly=true; 
paths=[]; #unless overridden we will convert this to a list of straight paths
defaultPrecision=1e-40;
precision=[]; #if empty we will use the default precision for all systems

#load("~/git-projects/periods/integrate/data/current.sage")
load("~/git-projects/period-suite/current.sage")
steps=len(allODEs);
cohomologies=[];
allTransMats=[1..steps];

if len(paths)==0:
    paths=[[0,1] for j in [1..steps]]
    
if len(precision)==0:
    precision=[defaultPrecision for j in [1..steps]]


t0=time.time();


print "Integrating systems", 1, "through", steps;
for j in [1..steps]:
    if periodsOnly and j==steps:
        noOfdeqs=geometricGenus;
    else:
        noOfdeqs=len(allODEs[j-1]);
    deqs=[deq.numerator() for deq in allODEs[j-1][0:noOfdeqs]];
    print "";
    print "Integrating system number", j;
    print "";
    %time transitionMatricies = [deqs[i-1].numerical_transition_matrix(paths[j-1], precision[j-1],assume_analytic=true) for i in [1..noOfdeqs]]
    allTransMats[j-1]=transitionMatricies;
    print "Maximal error in transition matricies:"
    print [max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies]
    cohomology=[1..noOfdeqs];
    for i in [1..noOfdeqs]:
        if j==1:
            init=Matrix(inits[i-1]);
            a = init.nrows(); b = init.ncols();
            init =  MatrixSpace(ComplexBallField(300), a, b)(init);
        else:
            init=Matrix(rawInits[j-2][i-1])*cohomologies[j-2];
        transitionMatrix=transitionMatricies[i-1];
        cohomology[i-1]=(transitionMatrix * init).row(0);
    cohomologies=cohomologies+[Matrix(cohomology)];
    
periodsWithError=Matrix(cohomologies[-1].rows()[0:geometricGenus]);
maximalError=max(periodsWithError.apply_map(lambda x : x.diameter()).list());
periods=periodsWithError.apply_map(lambda x : x.mid());
print "\nAccumulated maximal error:", maximalError
t1=time.time();
print "\nIt took", t1-t0, "seconds in total.\n"
print periods


Integrating systems 1 through 5

Integrating system number 1

CPU times: user 14.7 s, sys: 235 ms, total: 14.9 s
Wall time: 14.8 s
Maximal error in transition matricies:
[1.1148523e-42, 2.7313543e-41, 2.7313543e-41, 1.9775127e-42, 2.7313543e-41, 1.9775127e-42, 6.9664383e-42, 6.4651261e-42, 4.5861380e-42, 6.4651261e-42, 4.5861380e-42, 1.8597171e-41, 4.5861380e-42, 1.8597171e-41, 1.1148523e-42, 9.6978550e-42, 1.3269963e-41, 9.6978550e-42, 1.3269963e-41, 1.3269963e-41, 6.9664383e-42]

Integrating system number 2

CPU times: user 604 ms, sys: 35.3 ms, total: 639 ms
Wall time: 613 ms
Maximal error in transition matricies:
[1.2452203e-47, 1.3202322e-43, 1.6692524e-44, 1.6692524e-44, 3.3300315e-44, 3.3300315e-44, 3.3300315e-44, 3.9297500e-47, 3.0418406e-44, 3.9297500e-47, 3.0418406e-44, 1.7317783e-46, 3.0418406e-44, 1.7317783e-46, 1.2452203e-47, 2.9343524e-47, 2.9950129e-44, 2.9343524e-47, 2.9950129e-44, 2.9950129e-44, 1.3202322e-43]

Integrating system number 3

CPU times: user 3.07 s, sys: 

In [26]:
# Only for K3s
##
## Try a few values for "scale", do not exceed the negative exponent of accumulated error
##
scale = 30
##
##
periodVector=periods
u1 = periodVector.apply_map(real)[0].list()
u2 = periodVector.apply_map(imag)[0].list()
M = matrix([u1,u2])
#M = matrix([u1,u2])
#proj = (M.pseudoinverse() * M)
proj = M.transpose()
lattice = block_matrix([[(10^scale*proj).apply_map(lambda x : x.round()), matrix.identity(proj.dimensions()[0])]])
generatorNorms = [r.norm().n() for r in lattice.LLL().rows()]
consecutiveRatios = [generatorNorms[i]/generatorNorms[i-1] for i in [1..20]]

j=0; rank=1;
while j <= 19 :
    if consecutiveRatios[j] > 100 :
        j=20
    rank = rank + 1
    j=j+1
    
if generatorNorms[0] > 50 :
    rank = 1

rank

1

In [24]:
generatorNorms

[95.6660859448112,
 112.414411887444,
 115.602768132947,
 114.878196364671,
 135.558105622644,
 129.560796539694,
 130.518197964882,
 120.029163122968,
 121.453694879983,
 119.436175424366,
 134.420980505277,
 116.331423097975,
 130.326513035529,
 139.799856938410,
 140.110670542968,
 275646.804799185,
 280777.409953151,
 301846.503120377,
 300805.275071765,
 341003.930735410,
 336794.857582476]

In [25]:
lattice.LLL().row(0)

(7, 5, -32, -15, 28, -6, -41, 37, 1, -20, -24, -4, 4, -29, 1, 20, 26, 3, -18, 11, -23, -7, 0)